Name : Bhavyasai Chinchugalla
       
       High Performance Computing - 07
      
       Week 07 - Spark Application

Banner Id : 001321696

In [123]:
from pyspark.sql import SparkSession

# Create Spark session
spark = SparkSession.builder \
    .appName("OpenPaymentsAnalysis") \
    .getOrCreate()


In [127]:
import os

print("File exists:", os.path.exists("PGYR2023_P01302025_01212025.zip"))
print("File size:", os.path.getsize("PGYR2023_P01302025_01212025.zip"), "bytes")


File exists: True
File size: 789005271 bytes


In [128]:
with open("PGYR2023_P01302025_01212025.zip", "rb") as f:
    print(f.read(4))


b'PK\x03\x04'


In [129]:
import zipfile
import os

zip_path = "PGYR2023_P01302025_01212025.zip"
extract_dir = "openpayments"

# Create the directory if it doesn't exist
os.makedirs(extract_dir, exist_ok=True)

# Try unzipping again
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

# Check contents
os.listdir(extract_dir)


['OP_DTL_RSRCH_PGYR2023_P01302025_01212025.csv',
 'OP_REMOVED_DELETED_PGYR2023_P01302025_01212025.csv',
 'OP_DTL_GNRL_PGYR2023_P01302025_01212025.csv',
 'OP_DTL_OWNRSHP_PGYR2023_P01302025_01212025.csv',
 'OP_PGYR2023_README_P01302025.txt']

In [130]:
import zipfile
import os

zip_path = "PHPRFL_P01302025_01212025.zip"
extract_dir = "covered_recipients"

os.makedirs(extract_dir, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

# List the contents
os.listdir(extract_dir)


['OP_CVRD_RCPNT_PRFL_SPLMTL_README_P01302025.txt',
 'OP_CVRD_RCPNT_PRFL_SPLMTL_P01302025_01212025.csv']

In [131]:
import os
os.listdir("openpayments")
['OP_DTL_GNRL_PGYR2023_P01302025.csv']


['OP_DTL_GNRL_PGYR2023_P01302025.csv']

In [132]:
csv_path = "openpayments/OP_DTL_GNRL_PGYR2023_P01302025.csv"


In [133]:
import os
os.listdir("openpayments")


['OP_DTL_RSRCH_PGYR2023_P01302025_01212025.csv',
 'OP_REMOVED_DELETED_PGYR2023_P01302025_01212025.csv',
 'OP_DTL_GNRL_PGYR2023_P01302025_01212025.csv',
 'OP_DTL_OWNRSHP_PGYR2023_P01302025_01212025.csv',
 'OP_PGYR2023_README_P01302025.txt']

In [134]:
import os
os.listdir("covered_recipients")


['OP_CVRD_RCPNT_PRFL_SPLMTL_README_P01302025.txt',
 'OP_CVRD_RCPNT_PRFL_SPLMTL_P01302025_01212025.csv']

In [135]:
from pyspark.sql.functions import concat_ws, col

# Add full name if it's missing
joined_df = joined_df.withColumn(
    "Physician_Name",
    concat_ws(" ", col("Covered_Recipient_First_Name"), col("Covered_Recipient_Last_Name"))
)


In [139]:
preview_df = joined_df.select(
    "Covered_Recipient_Profile_ID",
    "Physician_Name",
    "Covered_Recipient_Specialty_1",
    "Recipient_State",
    "Total_Amount_of_Payment_USDollars",
    "Nature_of_Payment_or_Transfer_of_Value"
).limit(10)
preview_pd = preview_df.toPandas()
from tabulate import tabulate
print(tabulate(preview_pd, headers="keys", tablefmt="fancy_grid"))



╒════╤════════════════════════════════╤═══════════════════════╤══════════════════════════════════════════════════════════════════════════════════════════╤═══════════════════╤═════════════════════════════════════╤══════════════════════════════════════════╕
│    │   Covered_Recipient_Profile_ID │ Physician_Name        │ Covered_Recipient_Specialty_1                                                            │ Recipient_State   │   Total_Amount_of_Payment_USDollars │ Nature_of_Payment_or_Transfer_of_Value   │
╞════╪════════════════════════════════╪═══════════════════════╪══════════════════════════════════════════════════════════════════════════════════════════╪═══════════════════╪═════════════════════════════════════╪══════════════════════════════════════════╡
│  0 │                           3749 │ MICHAEL BIUNNO        │ Allopathic & Osteopathic Physicians|Psychiatry & Neurology|Psychiatry                    │ LA                │                               11.39 │ Food and Beverage  

Analysis 1: Nature of Payments > $1,000 by Count

In [140]:
from pyspark.sql.functions import col, count as _count
from tabulate import tabulate

# Step 1: Perform Analysis in Spark
analysis_1 = (
    joined_df
    .filter(col("Total_Amount_of_Payment_USDollars") > 1000)
    .na.drop(subset=["Nature_of_Payment_or_Transfer_of_Value"])
    .groupBy("Nature_of_Payment_or_Transfer_of_Value")
    .agg(_count("*").alias("count"))
    .orderBy(col("count").desc())
)

# Step 2: Convert to Pandas for clean table display
analysis_1_pd = analysis_1.toPandas()

# Show formatted table
from tabulate import tabulate
print(tabulate(analysis_1_pd, headers="keys", tablefmt="fancy_grid"))





╒════╤════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╤═════════╕
│    │ Nature_of_Payment_or_Transfer_of_Value                                                                                                             │   count │
╞════╪════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╪═════════╡
│  0 │ Consulting Fee                                                                                                                                     │     530 │
├────┼────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┼─────────┤
│  1 │ Compensation for services other than consulting, including serving as faculty or as a speaker at a venue other than a continuing education program │     189 │
├───

Analysis 2: Top 10 Nature of Payments by Count

In [141]:
from pyspark.sql.functions import count as _count, col
import pandas as pd
from IPython.display import display
import tabulate

# Analysis 2: Top 10 most frequent Nature of Payments
analysis_2 = (
    joined_df
    .na.drop(subset=["Nature_of_Payment_or_Transfer_of_Value"])
    .groupBy("Nature_of_Payment_or_Transfer_of_Value")
    .agg(_count("*").alias("count"))
    .orderBy(col("count").desc())
    .limit(10)
)

# Convert to Pandas for clean table display
analysis_2_pd = analysis_2.toPandas()

# Show formatted table
from tabulate import tabulate
print(tabulate(analysis_2_pd, headers="keys", tablefmt="fancy_grid"))


╒════╤════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╤═════════╕
│    │ Nature_of_Payment_or_Transfer_of_Value                                                                                                             │   count │
╞════╪════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╪═════════╡
│  0 │ Food and Beverage                                                                                                                                  │   95470 │
├────┼────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┼─────────┤
│  1 │ Compensation for services other than consulting, including serving as faculty or as a speaker at a venue other than a continuing education program │    1109 │
├───

Analysis 3: Top 10 Nature of Payments by Total Amount

In [142]:
from pyspark.sql.functions import sum as _sum, col
import pandas as pd
from IPython.display import display
import tabulate

# Analysis 3: Top 10 Nature of Payments by Total Amount
analysis_3 = (
    joined_df
    .na.drop(subset=["Nature_of_Payment_or_Transfer_of_Value", "Total_Amount_of_Payment_USDollars"])
    .groupBy("Nature_of_Payment_or_Transfer_of_Value")
    .agg(_sum("Total_Amount_of_Payment_USDollars").alias("total_amount"))
    .orderBy(col("total_amount").desc())
    .limit(10)
)

# Convert to Pandas DataFrame for display
analysis_3_pd = analysis_3.toPandas()

# Show formatted table
from tabulate import tabulate
print(tabulate(analysis_3_pd, headers="keys", tablefmt="fancy_grid"))


╒════╤════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╤══════════════════╕
│    │ Nature_of_Payment_or_Transfer_of_Value                                                                                                             │     total_amount │
╞════╪════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╪══════════════════╡
│  0 │ Consulting Fee                                                                                                                                     │      3.35086e+06 │
├────┼────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┼──────────────────┤
│  1 │ Royalty or License                                                                                                    

Analysis 4: Top 10 Specialties by Total Amount

In [143]:
from pyspark.sql.functions import sum as _sum, col
import pandas as pd
from IPython.display import display
import tabulate

# Analysis 4: Top 10 specialties by total payment amount
analysis_4 = (
    joined_df
    .na.drop(subset=["Covered_Recipient_Specialty_1", "Total_Amount_of_Payment_USDollars"])
    .groupBy("Covered_Recipient_Specialty_1")
    .agg(_sum("Total_Amount_of_Payment_USDollars").alias("total_amount"))
    .orderBy(col("total_amount").desc())
    .limit(10)
)

# Convert to Pandas DataFrame for pretty printing
analysis_4_pd = analysis_4.toPandas()

# Show formatted table
from tabulate import tabulate
print(tabulate(analysis_4_pd, headers="keys", tablefmt="fancy_grid"))


╒════╤══════════════════════════════════════════════════════════════════════════════════════════╤══════════════════╕
│    │ Covered_Recipient_Specialty_1                                                            │     total_amount │
╞════╪══════════════════════════════════════════════════════════════════════════════════════════╪══════════════════╡
│  0 │ Allopathic & Osteopathic Physicians|Orthopaedic Surgery                                  │      2.11226e+06 │
├────┼──────────────────────────────────────────────────────────────────────────────────────────┼──────────────────┤
│  1 │ Allopathic & Osteopathic Physicians|Pathology|Blood Banking & Transfusion Medicine       │      1.82558e+06 │
├────┼──────────────────────────────────────────────────────────────────────────────────────────┼──────────────────┤
│  2 │ Allopathic & Osteopathic Physicians|Orthopaedic Surgery|Orthopaedic Surgery of the Spine │ 752959           │
├────┼──────────────────────────────────────────────────────────

Analysis 5: Top 10 Physicians by Total Amount

In [144]:
from pyspark.sql.functions import concat_ws, sum as _sum, col
import pandas as pd
from IPython.display import display
import tabulate

# Create full physician name column
joined_df = joined_df.withColumn(
    "Physician_Name",
    concat_ws(" ", col("Covered_Recipient_First_Name"), col("Covered_Recipient_Last_Name"))
)

# Group by name and total payment
analysis_5 = (
    joined_df
    .filter(col("Physician_Name") != "")
    .groupBy("Physician_Name")
    .agg(_sum("Total_Amount_of_Payment_USDollars").alias("total_amount"))
    .orderBy(col("total_amount").desc())
    .limit(10)
)

# Convert to Pandas for pretty printing
analysis_5_pd = analysis_5.toPandas()

# Show formatted table
from tabulate import tabulate
print(tabulate(analysis_5_pd, headers="keys", tablefmt="fancy_grid"))


╒════╤════════════════════╤══════════════════╕
│    │ Physician_Name     │     total_amount │
╞════╪════════════════════╪══════════════════╡
│  0 │ Shawn ODriscoll    │      1.33381e+06 │
├────┼────────────────────┼──────────────────┤
│  1 │ Darrell Triulzi    │ 561711           │
├────┼────────────────────┼──────────────────┤
│  2 │ MICHAEL GOULD      │ 417266           │
├────┼────────────────────┼──────────────────┤
│  3 │ Lirong Qu          │ 385608           │
├────┼────────────────────┼──────────────────┤
│  4 │ Mark Yazer         │ 385261           │
├────┼────────────────────┼──────────────────┤
│  5 │ MARK BORIGINI      │ 347780           │
├────┼────────────────────┼──────────────────┤
│  6 │ Irina Chibisov     │ 337133           │
├────┼────────────────────┼──────────────────┤
│  7 │ Alesia Kaplan      │ 323518           │
├────┼────────────────────┼──────────────────┤
│  8 │ Sally Campbell-Lee │ 239896           │
├────┼────────────────────┼──────────────────┤
│  9 │ Suzann